In [1]:
from math import isclose
import pandas as pd
import numpy as np
import ast
import re

In [2]:
# đọc dữ liệu
df = pd.read_csv(r"D:\I. KhongPhanLoai\IT-job-analysis-VN\data_raw\ITViec_data.csv")

In [3]:
# xem thông tin cơ bản về bộ dữ liệu
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1159 entries, 0 to 1158
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   job_names             1159 non-null   object
 1   company_names         1159 non-null   object
 2   salaries              1159 non-null   object
 3   position_names        1159 non-null   object
 4   kind_jobs             1159 non-null   object
 5   array_skills          1159 non-null   object
 6   locate_names          1159 non-null   object
 7   exp_skills            1159 non-null   object
 8   domain_arr            1159 non-null   object
 9   post_dates_formatted  1159 non-null   object
dtypes: object(10)
memory usage: 90.7+ KB


bộ dữ liệu bao gồm 993 mẫu, 10 thuộc tính bao gồm:
- job_names: tên công việc
- company_names: tên công ty
- salaries: mức lương
- kind_job: loại công việc (toàn thời gian, bán thời gian, thực tập, vv)
- array_skills: các công cụ sử dụng
- locate_names: địa điểm làm việc
- exp_skills: kinh nghiêm và kĩ năng yêu cầu
- domain_arr: lĩnh vực công việc
- post_dates_formatted: ngày đăng tuyển
==> tất cả là biến phân loại trừ salary.

# KIỂM TRA NULL, NAN, DUPLICATE

In [4]:
# kiểm tra giá trị null
df.isnull().sum()

job_names               0
company_names           0
salaries                0
position_names          0
kind_jobs               0
array_skills            0
locate_names            0
exp_skills              0
domain_arr              0
post_dates_formatted    0
dtype: int64

In [5]:
# kiểm tra dữ liệu trùng
df[df.duplicated(keep=False)]

,job_names,company_names,salaries,position_names,kind_jobs,array_skills,locate_names,exp_skills,domain_arr,post_dates_formatted
19,Automation Tester,Simpson Strong-Tie Vietnam,You'll love it,Automation Tester,Hybrid (flexible between home and office),"['Automation Test', 'C#', 'QA QC', 'JavaScript...","9th Floor, Etown 6 Building, 364 Cong Hoa Stre...",Desired skills and experience: \nBachelor's de...,['Software Products and Web Services'],25/11/2025 17:12
20,Automation Tester,Simpson Strong-Tie Vietnam,You'll love it,Automation Tester,Hybrid (flexible between home and office),"['Automation Test', 'C#', 'QA QC', 'JavaScript...","9th Floor, Etown 6 Building, 364 Cong Hoa Stre...",Desired skills and experience: \nBachelor's de...,['Software Products and Web Services'],25/11/2025 17:12
26,Manual Tester (QA QC),Công ty cổ phần Tập đoàn Công nghệ Quảng Ích (...,13-20m,Manual Tester,At office,"['Tester', 'Mobile Apps', 'QA QC']","46 LePARC by Gamuda, Công Viên Yên Sở, Hoang M...",Có từ 2 năm trở lên kinh nghiệm Tester.\nNhanh...,['Education and Training'],25/11/2025 16:12
27,Manual Tester (QA QC),Công ty cổ phần Tập đoàn Công nghệ Quảng Ích (...,13-20m,Manual Tester,At office,"['Tester', 'Mobile Apps', 'QA QC']","46 LePARC by Gamuda, Công Viên Yên Sở, Hoang M...",Có từ 2 năm trở lên kinh nghiệm Tester.\nNhanh...,['Education and Training'],25/11/2025 16:12
57,"Senior Software Developer (C#, .Net, SQL)",MiTek Viet Nam,You'll love it,Fullstack Developer,At office,"['.NET', 'English', 'Azure', 'SQL', 'WinForms'...","Tòa nhà A5, Lô số A5, khu E-Office, đường Sáng...",Required Qualifications:\nBachelor of Science ...,['Software Products and Web Services'],25/11/2025 09:13
58,"Senior Software Developer (C#, .Net, SQL)",MiTek Viet Nam,You'll love it,Fullstack Developer,At office,"['.NET', 'English', 'Azure', 'SQL', 'WinForms'...","Tòa nhà A5, Lô số A5, khu E-Office, đường Sáng...",Required Qualifications:\nBachelor of Science ...,['Software Products and Web Services'],25/11/2025 09:13
60,Java Developer (3 years +),CODE88 COMPANY LIMITED,"1,000 - 2,500 USD",Backend Developer,At office,"['Spring', 'ReactJS', 'Spring Boot']","6th Floor, Menas Mall Saigon Airport, 60A Truo...",Skills & Qualifications:\nSoftware development...,['Financial Services'],25/11/2025 07:14
61,Java Developer (3 years +),CODE88 COMPANY LIMITED,"1,000 - 2,500 USD",Backend Developer,At office,"['Spring', 'ReactJS', 'Spring Boot']","6th Floor, Menas Mall Saigon Airport, 60A Truo...",Skills & Qualifications:\nSoftware development...,['Financial Services'],25/11/2025 07:14
63,Principal QA QC Engineer (Manual & Automation ...,"MONEY FORWARD VIETNAM CO.,LTD",You'll love it,Automation Tester,At office,"['QA QC', 'LLM', 'API', 'AI', 'A/B testing', '...","ETown Central Building, 11 Doan Van Bo, Ward 1...",Technical\nMust have\n5+ years of experience i...,"['Software Products and Web Services', 'Financ...",25/11/2025 00:14
64,Principal QA QC Engineer (Manual & Automation ...,"MONEY FORWARD VIETNAM CO.,LTD",You'll love it,Automation Tester,At office,"['QA QC', 'LLM', 'API', 'AI', 'A/B testing', '...","ETown Central Building, 11 Doan Van Bo, Ward 1...",Technical\nMust have\n5+ years of experience i...,"['Software Products and Web Services', 'Financ...",25/11/2025 00:14


In [6]:
# xóa dữ liệu trùng
df = df.drop_duplicates()

# CLEAN CỘT JOB_NAMES

In [7]:
# kiểm tra số lượng các job tương tự nhau
df.job_names.nunique()
# ==> có 961 job khác nhau ==> vô lý

1127

# đặc điểm của cột job_names:
- tên job bao gồm tên job, cấp bậc, công ty tuyển dụng, yêu cầu ngôn ngữ, công cụ sử dụng, loại hình công việc
- có nhiều job trùng tên nhưng có thêm loại hình làm việc
- có nhiều job trùng tên nhưng khác công ty tuyển dụng
- có nhiều job trùng tên nhưng khác yêu cầu ngôn ngữ, công cụ sử dụng
- có nhiều job trùng tên nhưng khác cấp bậc
- thậm chí có nhiều job không có tên công việc rõ ràng

* hướng giải quyết:
- giữ tên job chính, các thông tin về level, yêu cầu ngôn như, công cụ sử dụng hay loại hình công việc sẽ giữ lại để tiếp tục mổ xẻ để lấy thông tin tiếp
- các bước:
+ lấy job chính (thường là chỉ có tên job, không nằm trong ngoặc, không có dấu gạch ngang, không có dấu phẩy)
+ lọc và loại các yêu cầu level
* không được xóa hết thông tin mà chỉ đổ sang cột khác, nếu không có thì có thể cho null để tránh sai lệch dữ liệu

In [8]:
# lọc sửa clean lần 1

levels = ["intern","fresher","all level", "junior", "mid", "middle", "senior", "lead", "principal"]
pattern_level = r'(' + '|'.join(levels) + r')'

# 1. Tách level (không phân biệt hoa thường)
df["level"] = df["job_names"].str.lower().str.extract(pattern_level, expand=False)

# 2. Tách nội dung trong ngoặc → xu_ly_sau
df["xu_ly_sau"] = df["job_names"].str.extract(r'\((.*?)\)', expand=False)

# 3. Tạo cột job_names_clean
def clean_job_name(job):
    if pd.isna(job):
        return None
    # Loại bỏ dấu ngoặc và nội dung trong ngoặc
    job = re.sub(r'\(.*?\)', '', job)
    # Loại bỏ dấu gạch ngang và dấu phẩy
    job = re.sub(r'[-,]', '', job)
    # Loại bỏ level đã tách
    for level in levels:
        pattern = re.compile(level, flags=re.IGNORECASE)
        job = pattern.sub('', job)
    # Xóa khoảng trắng thừa ở đầu cuối
    job = job.strip()
    return job

df["job_names_clean"] = df["job_names"].apply(clean_job_name)

# Giữ nguyên index gốc
df = df.sort_index()


In [9]:
# lọc sửa clean lần 2

def clean_job_name_step4(job):
    if pd.isna(job):
        return pd.Series([None, None, None])  # ky_tu, xu_ly_sau2, job_names_clean2
    
    ky_tu = []
    xu_ly_sau2 = []

    # 1. Lấy nội dung trong [...] sang xu_ly_sau2
    bracket_matches = re.findall(r'\[(.*?)\]', job)
    if bracket_matches:
        xu_ly_sau2.extend(bracket_matches)
        for bm in bracket_matches:
            job = job.replace(f"[{bm}]", "")

    # 2. Tách số kèm "up to", "$", "M", "k" hoặc ~ sang ky_tu
    number_pattern = r'(?i)(up to\s*)?(\$)?(\d+\.?\d*)([kM\$~])?'
    number_matches = re.findall(number_pattern, job)
    if number_matches:
        for match in number_matches:
            full_match = ''.join(match).strip()
            if full_match:
                ky_tu.append(full_match)
                job = job.replace(full_match, '')

    # 3. Lấy dấu / nếu ở đầu chuỗi
    if job.startswith("/"):
        ky_tu.append("/")
        job = job[1:]

    # 4. Xóa dấu . hoặc / ở đầu chuỗi **trừ khi là .NET**
    if job.startswith(('.', '/')) and not job.lower().startswith('.net'):
        job = job.lstrip('./')

    # 5. Làm sạch khoảng trắng thừa
    job_clean2 = job.strip()

    ky_tu_str = " ".join(ky_tu) if ky_tu else None
    xu_ly_sau2_str = ", ".join(xu_ly_sau2) if xu_ly_sau2 else None

    return pd.Series([ky_tu_str, xu_ly_sau2_str, job_clean2])

# Áp dụng cho job_names_clean
df[['ky_tu', 'xu_ly_sau2', 'job_names_clean2']] = df['job_names_clean'].apply(clean_job_name_step4)

# Giữ nguyên index gốc
df = df.sort_index()


In [10]:
# Xóa cột job_names, job_names_clean vì không cần nữa
df = df.drop(['job_names', 'job_names_clean'], axis=1)

In [11]:
# đổi tên cột job_names sạch lại thành job_names
df = df.rename(columns={'job_names_clean2': 'job_names'})

# xử lý phân loại cho job_names

Được, tôi sẽ phân tích chi tiết cho bạn cách **gom các tên job khác nhau về nhóm công việc chung**, từ ý tưởng đến cách làm, ưu/nhược điểm, và cách đánh giá hiệu quả.

---

### **1. Mục tiêu**

Bạn có cột `job_names` với nhiều tên công việc khác nhau, nhưng thực tế nhiều tên chỉ khác nhau về chi tiết (ví dụ: `Java Developer`, `Java Backend Developer`, `Backend Java Developer`). Bạn muốn gom chúng về **nhóm chính**, ví dụ:

* Backend Developer
* Frontend Developer
* Fullstack Developer
* Data Scientist / AI Engineer
* QA / Tester
* Project Manager / Product Owner

---

### **2. Ý tưởng lọc và gom nhóm**

**Cơ sở để phân nhóm chính xác:**

1. **Keyword-based (dựa vào từ khóa)**

   * Xác định các từ khóa đặc trưng của nhóm công việc:

     * Backend: `backend`, `java`, `python`, `golang`, `c++`, `nodejs`
     * Frontend: `frontend`, `react`, `vue`, `angular`, `ui/ux`
     * Fullstack: `fullstack`
     * QA / Tester: `qa`, `tester`, `automation`, `manual`
     * AI / Data: `data`, `ai`, `machine learning`, `ml`, `data scientist`, `analyst`
     * DevOps / Cloud: `devops`, `cloud`, `sre`
     * Manager / Lead: `manager`, `lead`, `head`, `director`

   * Đây là **cách phổ biến và dễ làm**, cho hiệu quả cao nếu tập trung vào từ khóa quan trọng.

2. **Regex hoặc text preprocessing**

   * Chuyển tất cả về lowercase
   * Xóa các ký tự thừa, dấu câu, khoảng trắng
   * Tách các từ thành danh sách, kiểm tra sự xuất hiện của từ khóa

3. **Mapping manual + semi-automated**

   * Tạo dictionary mapping các job phổ biến → nhóm chính:

     ```python
     mapping = {
         'java': 'Backend Developer',
         'python': 'Backend Developer',
         'fullstack': 'Fullstack Developer',
         'frontend': 'Frontend Developer',
         'qa': 'QA / Tester',
         'data': 'Data / AI',
         'ai': 'Data / AI',
         'devops': 'DevOps / Cloud',
         'manager': 'Manager / Lead'
     }
     ```
   * Dùng thuật toán kiểm tra từng tên job và gán vào nhóm dựa trên từ khóa đầu tiên match.

4. **Text embedding / NLP (nâng cao)**

   * Nếu bạn muốn chính xác ~90%+, có thể dùng embedding (BERT, SentenceTransformer) để đo similarity giữa tên job và nhóm mẫu.
   * Ví dụ: tính cosine similarity giữa `Java Backend Developer` và nhóm mẫu `Backend Developer`.

---

### **3. Cách làm thực tế**

Một workflow điển hình:

1. **Tiền xử lý dữ liệu**

   ```python
   df['job_clean'] = df['job_names'].str.lower().str.replace(r'[^a-z0-9\s]', '', regex=True)
   ```

2. **Xây từ khóa cho nhóm**

   ```python
   job_groups = {
       'Backend Developer': ['backend', 'java', 'python', 'golang', 'c++', 'nodejs'],
       'Frontend Developer': ['frontend', 'react', 'vue', 'angular', 'ui ux'],
       'Fullstack Developer': ['fullstack'],
       'QA / Tester': ['qa', 'tester', 'automation', 'manual'],
       'Data / AI': ['data', 'ai', 'machine learning', 'ml', 'analyst', 'engineer'],
       'DevOps / Cloud': ['devops', 'cloud', 'sre'],
       'Manager / Lead': ['manager', 'lead', 'head', 'director']
   }
   ```

3. **Gom nhóm theo keyword**

   ```python
   def assign_group(job):
       for group, keywords in job_groups.items():
           if any(k in job for k in keywords):
               return group
       return 'Other'

   df['job_group'] = df['job_clean'].apply(assign_group)
   ```

4. **Kiểm tra kết quả**

   ```python
   df[['job_names', 'job_group']].head(20)
   ```

---

### **4. Thuận lợi / Bất lợi**

**Thuận lợi:**

* Dễ triển khai, không cần AI quá phức tạp.
* Có thể xử lý hàng nghìn job nhanh chóng.
* Tinh chỉnh keyword hoặc mapping nếu thấy nhiều lỗi.

**Bất lợi:**

* Nếu tên job quá dài, lộn xộn, nhiều ký tự thừa → đôi khi gán nhầm nhóm.
* Các job mới, lạ không có keyword → vào nhóm “Other”.
* Không nhận biết ngữ cảnh: `Java` đôi khi là Backend nhưng trong job đặc biệt có thể không phải.

---

### **5. Đánh giá hiệu quả**

Để đảm bảo **độ chính xác ~90%**:

1. Lấy **sample 100-200 job** và gán nhóm thủ công → so sánh với kết quả tự động.
2. Tính **accuracy**:
   [
   \text{Accuracy} = \frac{\text{Số job đúng nhóm}}{\text{Tổng số job sample}} \times 100%
   ]
3. Tinh chỉnh **từ khóa** cho các nhóm bị sai.
4. Lặp lại vài lần cho đến khi độ chính xác >90%.

---

Nếu bạn muốn, tôi có thể viết cho bạn **một đoạn code Python đầy đủ**, từ tiền xử lý → gán nhóm → kiểm tra hiệu quả ~90% cho dữ liệu job names bạn gửi.

Bạn có muốn tôi làm luôn không?


In [12]:
df['job_clean'] = df['job_names'].str.lower().str.replace(r'[^a-z0-9\s]', '', regex=True)

# 2. Định nghĩa các nhóm và từ khóa, ưu tiên theo thứ tự quan trọng
job_groups = {
    'Manager / Lead': ['manager', 'lead', 'head', 'director', 'owner', 'pm', 'project manager', 'product owner', 'deputy', 'erp manager'],
    'DevOps / Cloud': ['devops', 'cloud', 'sre', 'infrastructure', 'system engineer', 'database reliability'],
    'Data / AI': ['data', 'ai', 'machine learning', 'ml', 'data scientist', 'analyst', 'vision', 'ai engineer'],
    'Mobile Developer': ['ios', 'android', 'flutter', 'mobile'],
    'Fullstack Developer': ['fullstack', 'full stack', 'fullstack engineer', 'full stack engineer'],
    'Backend Developer': ['back end','backend', 'java', 'python', 'golang', 'c++', 'nodejs', 'c#', '.net', 'odoo', 'firmware'],
    'Frontend Developer': ['frontend', 'react', 'vue', 'angular', 'ui ux', 'ui/ux', 'reactjs', 'nextjs', 'unity'],
    'QA / Tester': ['kiểm thử','Kiểm thử','qa', 'tester', 'automation', 'manual', 'qc'],
    'Other': []
}

# 3. Hàm gán nhóm
def assign_group(job):
    for group in ['Manager / Lead', 'DevOps / Cloud', 'Data / AI', 'Mobile Developer', 
                  'Fullstack Developer', 'Backend Developer', 'Frontend Developer', 'QA / Tester', 'Other']:
        keywords = job_groups[group]
        if any(k in job for k in keywords):
            return group
    return 'Other'

df['job_group'] = df['job_clean'].apply(assign_group)

In [13]:
# kiểm tra chênh lệch giữa các nhóm
df.job_group.value_counts()

job_group
Other                  288
Data / AI              197
Backend Developer      184
QA / Tester            105
Manager / Lead         102
Fullstack Developer     88
Frontend Developer      72
DevOps / Cloud          62
Mobile Developer        46
Name: count, dtype: int64

In [14]:
# Xóa cột job_clean vì không cần nữa
df = df.drop([ 'job_clean'], axis=1)

# tiếp tục xử lý các cột phát sinh từ cột làm sạch ban đầu để tiếp tục lấy thông tin

In [15]:
# hàm lấy lương và chuẩn hóa 

# Tùy chỉnh tỉ giá VND -> USD
EXCHANGE = 24500.0

def _to_usd_from_vnd(vnd_amount, exchange=EXCHANGE):
    return vnd_amount / exchange

def _format_amount_usd(amount, as_k=False, round_to=1):
    """
    amount: số USD (float)
    as_k: nếu True -> format như $1.4K khi >=1000
    round_to: làm tròn thập phân khi dùng K
    """
    if as_k and amount >= 1000:
        val = round(amount/1000, round_to)
        # nếu val là số nguyên như 3.0 -> hiển thị 3
        if isclose(val, int(val)):
            val = int(val)
        return f"${val}K"
    else:
        amt = int(round(amount))
        return f"${amt}"

def parse_salary_to_usd(text, exchange=EXCHANGE, as_k=False):
    """
    Trả về dạng chuỗi "Up to $<x>" hoặc "Unknown".
    """
    if text is None:
        return "Unknown"
    s = str(text).strip()

    # chuẩn hóa dấu gạch, dấu thập phân, space
    s_clean = s.replace("–", "-").replace("—", "-")
    s_low = s_clean.lower()

    # những từ chỉ văn bản -> Unknown
    textual = ["attractive", "competitive", "negotiable", "discuss", "love", "package", "think", "will"]
    if all(word in s_low for word in ["you", "will"]) or any(w in s_low for w in textual):
        # chú ý: một vài chuỗi có cả text + số (vd "Very attractive!!!") -> xử lý ở dưới nếu có số
        if not re.search(r'\d', s_low):
            return "Unknown"

    # ---> helper để parse số có dấu phẩy/dấu chấm phân cách nghìn
    def parse_number_like(x):
        # loại bỏ spaces, giữ dấu chấm và dấu phẩy
        t = x.strip()
        # nếu dạng 1.200.000 hoặc 25.000.000 => hiểu là VNĐ với dấu chấm phân nghìn
        if re.match(r'^\d{1,3}(\.\d{3})+(\,\d+)?$', t):
            return int(t.replace(".", "").replace(",", "")), "int"
        # nếu chứa cả dấu phẩy và dấu chấm: "1,200 - 1,400" -> coi dấu phẩy là 1000 separator
        t2 = t.replace(",", "")
        try:
            if '.' in t2:
                # có thể là decimal: 1.5K, 1.8m etc.
                return float(t2), "float"
            return int(t2), "int"
        except:
            return None, None

    # 1) Nếu có range a - b: lấy b (max)
    range_match = re.search(r'(\d[\d\.,]*)\s*[-–]\s*(\d[\d\.,]*)', s)
    if range_match:
        a_raw, b_raw = range_match.group(1), range_match.group(2)
        # tìm unit context (usd, vnd, triệu, tr, m, k, year, /year)
        unit_context = s_low
        # parse b
        parsed_b, typ = parse_number_like(b_raw)
        if parsed_b is None:
            # fallback parse a
            parsed_b, typ = parse_number_like(a_raw)
            if parsed_b is None:
                return "Unknown"

        # -------- nếu unit explicit là USD
        if "usd" in unit_context or "$" in unit_context:
            # K xử lý: nếu b_raw k hoặc contains 'k'
            if re.search(r'[kK]\b', b_raw):
                val_usd = float(parsed_b) * 1000
            else:
                val_usd = float(parsed_b)
            # nếu ghi year -> chia 12
            if re.search(r'per\s*year|/year|year\b', unit_context):
                val_usd = val_usd / 12.0
            return "Up to " + _format_amount_usd(val_usd, as_k=as_k)

        # -------- nếu unit explicit là VND (triệu, tr, vnd, vnđ, đ)
        if any(k in unit_context for k in ["vnd", "vnđ", "triệu", "triue", "tr ", "tr$", "đ", "triệu", "million vnd"]):
            # Nếu b_raw là dạng như 25.000.000 hoặc 25.000.000 - parse_number_like xử lý
            # nếu b_raw nhỏ (vd 25) và có 'triệu' hoặc 'm' thì hiểu 25 triệu
            if re.search(r'(triệu|tr\b|m\b)', s_low):
                # nếu b_raw như '28' cùng context 'triệu' -> 28 triệu VND
                # kiểm tra nếu b_raw >= 1000 thì coi là VND trực tiếp
                if typ == "int" and parsed_b < 1000:
                    vnd_amount = parsed_b * 1_000_000
                else:
                    # nếu b_raw đã là số lớn (25.000.000) -> parsed_b is int of full VND
                    vnd_amount = parsed_b
            else:
                # fallback: nếu parsed_b > 100000 => coi là VND
                if parsed_b > 100000:
                    vnd_amount = parsed_b
                else:
                    # không rõ -> Unknown
                    return "Unknown"
            usd = _to_usd_from_vnd(vnd_amount, exchange)
            return "Up to " + _format_amount_usd(usd, as_k=as_k)

        # -------- nếu không có unit rõ ràng
        #  nếu b_raw >= 100000 => nhiều khả năng VND
        if typ == "int" and parsed_b >= 100000:
            usd = _to_usd_from_vnd(parsed_b, exchange)
            return "Up to " + _format_amount_usd(usd, as_k=as_k)
        # nếu b_raw có k -> thousand USD
        if re.search(r'[kK]\b', b_raw):
            val_usd = float(parsed_b) * 1000
            return "Up to " + _format_amount_usd(val_usd, as_k=as_k)
        # ngược lại mặc định hiểu là USD (thousands if >=1000)
        val_usd = float(parsed_b)
        return "Up to " + _format_amount_usd(val_usd, as_k=as_k)

    # 2) Kiểm tra "up to", "upto", "from", "$..." đơn lẻ
    # Up to X
    m_upto = re.search(r'up\s*to\s*[\$]?\s*([\d\.,]+)\s*([kKmM]?)(.*)', s_low, flags=re.IGNORECASE)
    if m_upto:
        num_raw = m_upto.group(1)
        suffix = m_upto.group(2)
        tail = m_upto.group(3)  # để xem có 'vnd' hay 'year'
        parsed, typ = parse_number_like(num_raw)
        if parsed is None:
            return "Unknown"
        # nếu suffix k
        if suffix.lower() == 'k':
            usd = float(parsed) * 1000
            if 'year' in tail:
                usd = usd / 12.0
            return "Up to " + _format_amount_usd(usd, as_k=as_k)
        if suffix.lower() == 'm':
            # chú ý: nếu có 'vnd' xuất hiện hoặc 'triệu' -> hiểu M là triệu VND
            if any(k in s_low for k in ["vnd", "vnđ", "triệu", "tr ", "đ"]):
                vnd = float(parsed) * 1_000_000
                usd = _to_usd_from_vnd(vnd, exchange)
                return "Up to " + _format_amount_usd(usd, as_k=as_k)
            else:
                # nếu có $ hoặc usd -> triệu USD
                if "$" in s or "usd" in s_low:
                    usd = float(parsed) * 1_000_000
                    if 'year' in tail:
                        usd = usd / 12.0
                    return "Up to " + _format_amount_usd(usd, as_k=as_k)
                # nếu không rõ -> coi là million VND (an toàn)
                vnd = float(parsed) * 1_000_000
                usd = _to_usd_from_vnd(vnd, exchange)
                return "Up to " + _format_amount_usd(usd, as_k=as_k)

        # nếu không có suffix
        # xem context có vnd
        if any(k in s_low for k in ["vnd", "vnđ", "triệu", "tr ", "đ"]):
            # nếu num nhỏ (<1000) coi là triệu VND? có thể là 25 => 25 triệu
            if parsed < 1000:
                vnd = parsed * 1_000_000
            else:
                vnd = parsed
            usd = _to_usd_from_vnd(vnd, exchange)
            if 'year' in tail:
                usd = usd / 12.0
            return "Up to " + _format_amount_usd(usd, as_k=as_k)
        # USD context?
        if "$" in s or "usd" in s_low:
            usd = float(parsed)
            if 'year' in s_low:
                usd = usd / 12.0
            return "Up to " + _format_amount_usd(usd, as_k=as_k)
        # fallback: if number big -> VND else USD
        if parsed >= 100000:
            usd = _to_usd_from_vnd(parsed, exchange)
            return "Up to " + _format_amount_usd(usd, as_k=as_k)
        return "Up to " + _format_amount_usd(float(parsed), as_k=as_k)

    # 3) Kiểm tra dạng đơn lẻ có $ hoặc usd
    m_dollar = re.search(r'\$([\d\.,]+)\s*([kKmM]?)', s)
    if m_dollar:
        num_raw = m_dollar.group(1)
        suf = m_dollar.group(2)
        parsed, typ = parse_number_like(num_raw)
        if parsed is None:
            return "Unknown"
        if suf.lower() == 'k':
            usd = float(parsed) * 1000
        elif suf.lower() == 'm':
            usd = float(parsed) * 1_000_000
        else:
            usd = float(parsed)
        # nếu có 'year' thì chia 12
        if 'year' in s_low:
            usd = usd / 12.0
        return "Up to " + _format_amount_usd(usd, as_k=as_k)

    # 4) Kiểm tra VND dạng như '45.000.000', '25.000.000 - 40.000.000', '30m', '30 triệu'
    # dạng single number with dots -> coi là VND
    if re.match(r'^\d{1,3}(\.\d{3})+(\,\d+)?$', s_low) or re.search(r'(\d+)\s*(triệu|tr|vnđ|vnd|đ)', s_low):
        # lấy số lớn nhất nếu range xuất hiện (đã xử lý range trước)
        # chuyển về số nguyên VND
        # nếu có 'triệu' hoặc 'm' => tính triệu
        m_tri = re.search(r'([\d\.,]+)\s*(triệu|tr|m)\b', s_low)
        if m_tri:
            num_raw = m_tri.group(1)
            # parse like "28" or "28.000.000"
            parsed, typ = parse_number_like(num_raw)
            if parsed is None:
                return "Unknown"
            # nếu parsed nhỏ (<1000) coi là số triệu
            if parsed < 1000:
                vnd = parsed * 1_000_000
            else:
                vnd = parsed
            usd = _to_usd_from_vnd(vnd, exchange)
            return "Up to " + _format_amount_usd(usd, as_k=as_k)

        # nếu dạng 45.000.000
        num_parsed, _ = parse_number_like(s_low)
        if num_parsed is not None:
            # nếu lớn hơn 100k -> coi là VND full number
            if num_parsed >= 100000:
                usd = _to_usd_from_vnd(num_parsed, exchange)
                return "Up to " + _format_amount_usd(usd, as_k=as_k)

    # 5) Dạng xxxk (chỉ số + k) mà không có $ -> coi là thousand USD
    m_k = re.search(r'([\d\.,]+)\s*[kK]\b', s)
    if m_k:
        parsed, _ = parse_number_like(m_k.group(1))
        if parsed is None:
            return "Unknown"
        usd = float(parsed) * 1000
        return "Up to " + _format_amount_usd(usd, as_k=as_k)

    # 6) Dạng chỉ có 1 số nhỏ (<1000) mà có context 'usd' -> USD
    m_only_num = re.search(r'^\s*([\d\.,]+)\s*$', s)
    if m_only_num:
        parsed, _ = parse_number_like(m_only_num.group(1))
        if parsed is None:
            return "Unknown"
        if "usd" in s_low or "$" in s:
            usd = float(parsed)
            return "Up to " + _format_amount_usd(usd, as_k=as_k)
        # nếu số lớn thì có thể là VND
        if parsed >= 100000:
            usd = _to_usd_from_vnd(parsed, exchange)
            return "Up to " + _format_amount_usd(usd, as_k=as_k)
        # nếu nhỏ vô định -> Unknown
        return "Unknown"

    return "Unknown"

In [16]:
# lấy lương và chuẩn hóa từ 2 cột cần thiết

df['salary_clean2'] = df['salaries'].apply(lambda x: parse_salary_to_usd(x, exchange=EXCHANGE, as_k=False))
df['salary_clean1'] = df['ky_tu'].apply(lambda x: parse_salary_to_usd(x, exchange=EXCHANGE, as_k=False))

In [17]:
# tạo cột lương sạch

# Tạo cột final_salary
df['final_salary'] = np.where(
    (df['salary_clean2'].notna()) & (df['salary_clean2'] != 'Unknown'),
    df['salary_clean2'],   # ưu tiên salary_clean2 nếu khác 'Unknown'
    df['salary_clean1']    # nếu salary_clean2 là 'Unknown' thì dùng salary_clean1
)

In [18]:
df[["salaries","salary_clean1","salary_clean2","final_salary"]]

,salaries,salary_clean1,salary_clean2,final_salary
0,Up to 40m vnd,Unknown,Up to $1633,Up to $1633
1,You'll love it,Unknown,Unknown,Unknown
2,You'll love it,Unknown,Unknown,Unknown
3,You'll love it,Unknown,Unknown,Unknown
4,You'll love it,Unknown,Unknown,Unknown
...,...,...,...,...
1154,You'll love it,Unknown,Unknown,Unknown
1155,You'll love it,Unknown,Unknown,Unknown
1156,You'll love it,Unknown,Unknown,Unknown
1157,You'll love it,Unknown,Unknown,Unknown


In [19]:
# xóa các cột salaries, salary_clean1, salary_clean2, ky_tu
# Xóa cột job_names, job_names_clean vì không cần nữa
df = df.drop(['salaries', 'salary_clean1', 'salary_clean2','ky_tu'], axis=1)

In [20]:
# đổi tên cột job_names sạch lại thành job_names
df = df.rename(columns={'final_salary': 'salary'})

# hoàn thành xong làm sạch lương - salary

In [21]:
# xóa 2 cột không cần thiết nữa sau khi đã kiểm tra 

df = df.drop(['xu_ly_sau2'], axis=1)
df = df.drop(['xu_ly_sau'], axis=1)

In [22]:
# chuản hóa lại cho đồng bộ côt job_kinds

df['kind_jobs'] = df['kind_jobs'].replace({
    'At office': 'At office',
    'Hybrid (flexible between home and office)': 'Hybrid',
    "Remote (don't have to come to the office)": 'Remote'
})

In [23]:
# xóa cột job_names vì không cần nữa

df = df.drop(['job_names'], axis=1)

In [24]:
# phân nhóm theo job_groups

job_groups = [
    'Backend Developer', 'QA / Tester', 'Data / AI',
    'Fullstack Developer', 'DevOps / Cloud', 'Other',
    'Manager / Lead', 'Frontend Developer', 'Mobile Developer'
]

# Tạo dictionary mapping dựa trên từ khóa
keyword_to_group = {
    'game': 'Game',
    'backend': 'Backend Developer',
    'application': 'Backend Developer',
    'front': 'Frontend Developer',  # Frontend / Front
    'fullstack': 'Fullstack Developer',
    'tester': 'QA / Tester',        # Manual Tester, Automation Tester,...
    'qa': 'QA / Tester',
    'test': 'QA / Tester',
    'ai': 'Data / AI',              # AI, Machine Learning
    'machine learning': 'Data / AI',
    'data': 'Data / AI',            # Data Engineer, Data Scientist,...
    'bi':'Data / AI',
    'devops': 'DevOps / Cloud',
    'cloud': 'DevOps / Cloud',
    'mobile': 'Mobile Developer',
    'manager': 'Manager / Lead',
    'business analyst' : 'Manager / Lead',
    'architect': 'Manager / Lead',
    'product owner' : 'Manager / Lead',
    'scrum': 'Manager / Lead',
    'alige': 'Manager / Lead',
    'lead': 'Manager / Lead',
    'embedded engineer': 'Embedded / Firmware',
    'firmware engineer': 'Embedded / Firmware',
    'erp': 'ERP / Enterprise Systems',
    'ux/ui':'UX/UI Designer',
    'security': 'Security / Cybersecurity'
}

def assign_job_group(position_name):
    pos_lower = str(position_name).lower()
    for keyword, group in keyword_to_group.items():
        if keyword in pos_lower:
            return group
    return 'Other'  # nếu không match từ khóa nào

df['job_group'] = df['position_names'].apply(assign_job_group)

In [25]:
# kiểm tra theo nhóm

df.loc[df["job_group"] == 'Other', ['job_group', 'position_names']]

,job_group,position_names
24,Other,"C-level (CTO, CIO, CISO, CDO)"
93,Other,Systems Engineer / Administrator
106,Other,DevSecOps Engineer
127,Other,SysOps Engineer
137,Other,DevSecOps Engineer
...,...,...
1110,Other,Bridge System Engineer (BrSE)
1119,Other,Vice President / Director
1123,Other,Systems Engineer / Administrator
1154,Other,"C-level (CTO, CIO, CISO, CDO)"


In [26]:
# gom nhóm theo domain

# mapping domain → nhóm lớn
domain_map = {
    'Banking': 'Finance',
    'Financial Services': 'Finance',
    'Securities & Investment': 'Finance',
    'Insurance': 'Finance',
    
    'E-commerce': 'Ecommerce & Retail',
    'Retail and Wholesale': 'Ecommerce & Retail',
    'Trading and Commercial': 'Ecommerce & Retail',
    'Consumer Goods': 'Ecommerce & Retail',
    'Apparel and Fashion': 'Ecommerce & Retail',
    
    'IT Services and IT Consulting': 'IT & Software',
    'Software Development Outsourcing': 'IT & Software',
    'Software Products and Web Services': 'IT & Software',
    'IT Hardware and Computing': 'IT & Software',
    'Network and Infrastructure': 'IT & Software',
    'Cyber Security': 'IT & Software',
    
    'AI Software & Services': 'AI / Emerging Tech',
    'Emerging Tech R&D': 'AI / Emerging Tech',
    'Blockchain & Web3 Services': 'AI / Emerging Tech',
    
    'Game': 'Gaming',
    
    'Manufacturing and Engineering': 'Engineering / Manufacturing',
    'Materials and Mining': 'Engineering / Manufacturing',
    
    'Transportation, Logistics and Warehouse': 'Logistics',
    
    'Healthcare': 'Healthcare',
    'Pharmaceuticals': 'Healthcare',
    
    'Education and Training': 'Education / NGO',
    'Non-Profit and Social Services': 'Education / NGO',
    
    'Tourism and Hospitality Services': 'Hospitality / F&B',
    'Food and Beverage': 'Hospitality / F&B',
    
    'Media, Advertising and Entertainment': 'Media / Creative',
    'Creative and Design': 'Media / Creative',
    
    'Real Estate, Property and Construction': 'Real Estate / Construction',
    'Facility Management': 'Real Estate / Construction',
    'Utilities': 'Real Estate / Construction',
    
    'Telecommunication': 'Telecom',
    
    'Professional Services': 'Professional / Services',
    'Research Services': 'Professional / Services',
    'Staffing and Recruiting': 'Professional / Services',
    
    'Environment': 'Environment / Agriculture',
    'Agriculture': 'Environment / Agriculture',
    
    'Government': 'Government'
}

# 1. Chuyển chuỗi → list
df["domain_list"] = df["domain_arr"].apply(ast.literal_eval)

# 2. Map từng item trong list → nhóm lớn
def map_domains(domains):
    mapped = [domain_map.get(d, "Other") for d in domains]
    return list(set(mapped))  # bỏ trùng

df["domain_group"] = df["domain_list"].apply(map_domains)

In [27]:
# tạo một bảng dùng để phân nhóm domain

df_exploded_domain = df.explode("domain_group")

In [28]:
# Thống kê số lần xuất hiện của từng domain

domain_counts = df_exploded_domain['domain_group'].value_counts()
print(domain_counts.head(20))  # 20 kỹ năng phổ biến nhất

domain_group
IT & Software                  750
Finance                        307
AI / Emerging Tech             205
Ecommerce & Retail             163
Logistics                       52
Healthcare                      46
Gaming                          41
Real Estate / Construction      36
Professional / Services         30
Education / NGO                 29
Media / Creative                28
Hospitality / F&B               28
Engineering / Manufacturing     22
Telecom                         20
Other                           11
Government                       7
Environment / Agriculture        5
Name: count, dtype: int64


In [29]:
# xóa cột domain_arr

df = df.drop(['domain_arr'], axis=1)

In [30]:
# Chuyển sang datetime

df['post_date'] = pd.to_datetime(df['post_dates_formatted'], format='%d/%m/%Y %H:%M').dt.date

# Kiểm tra kết quả
print(df['post_date'].head())
print(type(df['post_date'][0]))

0    2025-11-25
1    2025-11-25
2    2025-11-25
3    2025-11-25
4    2025-11-25
Name: post_date, dtype: object
<class 'datetime.date'>


In [31]:
# xóa cột post_dates_formatted

df = df.drop(['post_dates_formatted'], axis=1)

In [32]:
# Tách lấy phần sau dấu ',' cuối cùng, bỏ khoảng trắng 2 bên

df['city'] = df['locate_names'].str.split(',').str[-1].str.strip()

In [33]:
# Chuẩn hóa city với 3 nhóm: Ha Noi, Ho Chi Minh, Da Nang

df['city_clean'] = df['city'].apply(lambda x: x if x in ['Ha Noi', 'Ho Chi Minh', 'Da Nang'] else 'Other')

In [34]:
# xóa 2 cột không cần sài nữa

df = df.drop(['locate_names','city'], axis=1)

In [35]:
# Chuyển string list thành list thật

df['skills_list'] = df['array_skills'].apply(ast.literal_eval)

In [36]:
# Explode mỗi skill thành 1 dòng

df_exploded_skills = df.explode('skills_list')

In [37]:
# Thống kê số lần xuất hiện của từng skill

skill_counts = df_exploded_skills['skills_list'].value_counts()
print(skill_counts.head(20))  # 20 kỹ năng phổ biến nhất


skills_list
English              293
Python               216
Java                 213
SQL                  178
JavaScript           156
ReactJS              136
AWS                  123
Agile                119
AI                   113
NodeJS               108
TypeScript           101
QA QC                 95
Tester                89
API                   87
Cloud                 87
Business Analysis     83
C#                    81
DevOps                79
Database              78
Automation Test       77
Name: count, dtype: int64


In [38]:
# xóa cột array_skills không cần sài nữa

df = df.drop(['array_skills'], axis=1)

In [39]:
# # lấy thông tin level từ cột level đổ vào level_mapped

# # Pattern mapping level
# level_patterns = {
#     'intern': r'1 years of|intern|internship|student|fresher|0 years|0\+|0-1|0-2|0–1|0–2|ít nhất 0 năm|minimum 0 years|at least 0 years|fresh|beginner|new graduate',
    
#     'junior': r'more than 1|có 1 năm|tối thiểu 01|1\+|1 - 3|từ 01 năm|tối thiểu 1 năm|từ 1 năm|junior|0-1 years|0-2 years|0–2|1-2|1–2|ít nhất 1|minimum of 2|minimum 1 years|at least 1|2 năm|từ 2 năm|ít nhất 2 năm|1-3|1–3|newbie|entry level',
    
#     'mid': r'3 years|minimum of 3|3\-5|3-4|more than 3|more than 2|3\s-\s5|từ 4|2\+|2-4|3-6|2\-3|3\s-\s4|từ 02 năm|từ 03 năm|trên 2 năm|at least 03|tối thiểu 3 năm|3\+|từ 3 năm|tối thiểu 4 năm|từ 2 năm|2-5|2–5|at least 4 years|mid|2\+ years|2-3|3-4|3-5|1–3|ít nhất 2 năm|ít nhất 3 năm|2–4|minimum 2 years|minimum 3 years|minimum 4 years|at least 2 years|at least 3 years|at least 4 years|intermediate|average experience',
    
#     'senior': r'4-5|5\+|tối thiểu 5 năm|trên 4 năm|trên 5 năm|từ 5 năm|senior|5-6|5-8|5–8|6\+|strong experience|4–6|4\+|ít nhất 4 năm|ít nhất 5 năm|minimum 5 years|at least 5 years|7 năm|từ 7 năm|experienced|advanced',
    
#     'lead': r'minimum of 7|từ 8 năm|lead|team lead|manager|ít nhất 6 năm|minimum 6 years|at least 6 years|6\+|6-7|8\+|8-10|ít nhất 8 năm|senior+|leadership|head|managerial',
    
#     'principal': r'principal|architect|expert|ít nhất 8 năm|minimum 8 years|at least 8 years|8\+|8-10|specialist|guru|expert level'
# }

# def map_level_from_skills(level_existing, exp_text):
#     if pd.notna(level_existing) and level_existing.lower() != 'all level':
#         return level_existing.capitalize()  # giữ level có sẵn
#     exp_lower = str(exp_text).lower()
#     for lvl, pattern in level_patterns.items():
#         if re.search(pattern, exp_lower):
#             return lvl
#     return 'Unknown'
# df['level_mapped'] = df.apply(lambda x: map_level_from_skills(x['level'], x['exp_skills']), axis=1)

# # Kiểm tra kết quả
# print(df['level_mapped'].value_counts())

'''========== ở trên là phiên bản đầu tiên'''

# ví dụ level_patterns (các patterns vẫn dùng làm fallback)
level_patterns = {
    'intern': r'\b(intern|internship|student|fresher|ít nhất 0 năm|minimum 0 years|at least 0 years|fresh|beginner|new graduate)\b',
    'junior': r'\b(junior|newbie|entry level|ít nhất 1|tối thiểu 1 năm)\b',
    'mid': r'\b(mid|intermediate|average experience|ít nhất 2 năm|tối thiểu 3 năm|3 years|2-4|2–4)\b',
    'senior': r'\b(senior|experienced|advanced|tối thiểu 5 năm|ít nhất 4 năm|at least 5 years)\b',
    'lead': r'\b(lead|team lead|manager|leadership|head|managerial|ít nhất 6 năm|at least 6 years)\b',
    'principal': r'\b(principal|architect|expert|specialist|guru|expert level|ít nhất 8 năm|at least 8 years)\b'
}

def map_level_from_skills(level_existing, exp_text):
    # giữ level có sẵn nếu khác All level
    if pd.notna(level_existing) and str(level_existing).strip().lower() != 'all level':
        return str(level_existing).strip().capitalize()

    text = str(exp_text).lower()

    # chuẩn hoá dấu gạch (hyphen) để regex dễ bắt
    text = re.sub(r'[–—−]', '-', text)

    # 1) thử extract số năm rõ ràng
    # sẽ match 0, 0.5, 1, 2+, "3-5", "3 - 5", "3 năm", "2 năm"
    range_match = re.search(r'(\d+(?:\.\d+)?)(?:\s*(?:\+|-|–|to)\s*(\d+(?:\.\d+)?))?\s*(?:years?|yrs?|năm)\b', text)
    plus_match = re.search(r'(\d+(?:\.\d+)?)\s*\+', text)  # 2+, 3+

    years = None
    if range_match:
        a = float(range_match.group(1))
        b = range_match.group(2)
        if b:
            b = float(b)
            years = (a + b) / 2.0  # trung bình khoảng để mapping
        else:
            years = a
    elif plus_match:
        years = float(plus_match.group(1))

    # mapping theo years nếu có
    if years is not None:
        if years <= 0:
            return 'intern'
        if years > 0 and years < 2:   # (0,2) => junior
            return 'junior'
        if 2 <= years < 4:
            return 'mid'
        if 4 <= years < 7:
            return 'senior'
        if years >= 7:
            return 'lead'
    
    # 2) fallback: thử regex keyword (theo thứ tự cụ thể: principal -> lead -> senior -> mid -> junior -> intern)
    # chú ý: so sánh theo thứ tự từ cao xuống thấp để tránh match rộng gây map sai
    order = ['principal', 'lead', 'senior', 'mid', 'junior', 'intern']
    for lvl in order:
        pattern = level_patterns.get(lvl)
        if pattern and re.search(pattern, text, flags=re.IGNORECASE):
            return lvl

    return 'Unknown'

# áp dụng lên df
df['level_mapped'] = df.apply(lambda x: map_level_from_skills(x.get('level', np.nan), x.get('exp_skills', '')), axis=1)

print(df['level_mapped'].value_counts())


level_mapped
mid          316
Senior       241
senior       163
Unknown      100
Mid           76
junior        67
Lead          65
lead          47
Junior        38
Fresher       17
Principal      8
principal      4
intern         2
Name: count, dtype: int64


In [40]:
# kiểm tra theo nhóm

df.loc[df["level_mapped"] == 'intern', ['exp_skills', 'level_mapped','level']]

,exp_skills,level_mapped,level
437,◆ Yêu cầu bắt buộc : \nBiết ngôn ngữ lập trình...,intern,NaN
939,Bachelor’s degree in Electronics and Engineeri...,intern,NaN


In [41]:
# join 2 bảng level lại với nhau

df['final_level'] = np.where(
    df['level'].notna(),    # nếu level khác null
    df['level'],            # giữ nguyên level
    df['level_mapped']      # nếu level null thì lấy từ level_mapped
)

In [42]:
# xóa 3 cột không sài nữa

df = df.drop(['level_mapped','level','exp_skills'], axis=1)

In [43]:
# đổi tên cột final_level sạch lại thành level

df = df.rename(columns={'final_level': 'level'})

In [44]:
'''================================================= tiếp tục xử lý cột skills_list=================================================================================================='''

'================================================= tiếp tục xử lý cột skills_list=================================================================================================='

In [45]:
# hàm phân loại nháp

data = df.skills_list

# ====== DANH SÁCH PHÂN LOẠI (chuyển về lowercase để so sánh) ======
tools = {t.lower() for t in [
    'API','Postman', 'Jira', 'Figma', 'Power BI', 'Firebase', 'AWS', 'Kubernetes',
    'Git', 'GitHub', 'GitLab', 'Docker', 'Jenkins', 'Terraform', 'Tableau', 'Metabase',
    'SAP', 'SAP BusinessObjects', 'ServiceNow', 'Redis', 'MongoDB', 'Oracle', 'MySQL',
    'MariaDB', 'PostgreSQL', 'Microsoft SQL Server', 'Google BigQuery', 'Apache Airflow',
    'Databricks', 'Elasticsearch', 'VMware', 'Hyper-V', 'OpenStack', 'Nginx', 'Cisco',
    'Juniper', 'Grafana', 'Splunk', 'WordPress', 'Shopify', 'WooCommerce', 'BigCommerce',
    'Adobe Photoshop', 'Adobe Illustrator', 'Adobe XD', 'Adobe', 'Miro', 'Sketch', 'AutoCAD',
    'Qt', 'Power Automate', 'Power Apps', 'VSCode', 'Android Studio', 'Jmeter', 'Zephyr',
    'SoapUI', 'Robot Framework', 'TestComplete', 'Cypress', 'Appium', 'Playwright',
    'Katalon', 'TestNG', 'JUnit', 'pytest', 'Retrofit', 'Kafka', 'Prometheus', 'InfluxDB',
    'Unreal Engine', 'Unity', 'Maya', 'Blender', 'Cocos', 'Live2D', '3ds Max', 'Phoenix',
    'Electron', 'Hyperledger Fabric', 'Salesforce', 'Azure', 'GCP', 'Azure SQL Database',
    'AWS RDS', 'AWS Lambda', 'GCP SQL', 'Bash Shell', 'PowerShell', 'Airtable',
    'GitHub Actions', 'Travis CI', 'IBM DB2'
]}

frameworks = {f.lower() for f in [
    '.NET', '.Net Core', 'ASP.NET', 'Entity Framework', 'Entity Framework Core', 'MVVM',
    'MVP', 'MVC', 'Spring', 'Spring Boot', 'Django', 'FastAPI', 'Flask', 'Laravel', 'Zend',
    'Symfony', 'CakePHP', 'Ruby on Rails', 'Angular', 'AngularJS', 'NextJS',
    'VueJS', 'NuxtJS', 'NestJS', 'Bootstrap', 'Tailwind', 'Gin', 'Express', 'ExpressJS',
    'Fiber', 'Keras', 'TensorFlow', 'PyTorch', 'OpenCV', 'Unity Framework (engine)',
    'Flutter', 'Ionic', 'WPF', 'UI5', 'Phoenix', 'Hibernate', 'J2EE', 'Rocket', 'Redux',
    'Jest', 'Cucumber', 'Grails', 'MFC', 'Retrofit', 'Soap', 'SOAP UI', 'OutSystems',
    'AUTOSAR', 'RTOS', 'ThreadX', 'Blazor', 'Microservices Architecture', 'REST Assured', 'React Native'
]}

libraries = {l.lower() for l in [
    'Spark', 'LINQ', 'NumPy', 'Pandas', 'OpenCV', 'Keras', 'TensorFlow', 'PyTorch',
    'jsonwebtoken', 'GraphQL', 'JUnit', 'Jest', 'pytest', 'Redux', 'kotlinx.coroutines',
    'Apache HttpClient', 'Requests', 'Axios', 'D3.js', 'Matplotlib', 'Seaborn', 'Plotly',
    'Bokeh', 'ggplot2', 'Leaflet', 'Three.js', 'Plotly.js', 'Chart.js', 'Highcharts',
    'ECharts', 'Vega-Lite','Selenium','JQuery','React','ReactJS'
]}

code_languages = {c.lower() for c in [
    'Python', 'Java', 'C++', 'C#', 'Ruby', 'PHP', 'Golang', 'Kotlin', 'Swift',
    'Objective-C', 'R', 'Rust', 'Scala', 'Dart', 'Elixir', 'Groovy', 'SQL', 'PL/SQL',
    'JavaScript', 'TypeScript', 'HTML', 'HTML5', 'CSS', 'CSS3', 'Embedded C', 'C language',
    'ABAP', 'vb.net', 'Fortran', 'COBOL', 'MATLAB', 'Shell scripting', 'PowerShell',
    'Perl', 'Lua', 'Haskell', 'Erlang', 'Julia', 'F#', 'Ada', 'Lisp', 'Prolog','CSS 3'
]}

languages = {c.lower() for c in [
    'English', 'Japanese','Korean','French','Chinese'
]}


# ====== TIẾN HÀNH XỬ LÝ ======

seen = set()
unique_items = []

# loại trùng (không phân biệt hoa/thường)
for row in data:
    for item in row:
        key = item.lower()       # chuyển về chữ thường để kiểm tra
        if key not in seen:
            unique_items.append(item)  # giữ lại giá trị gốc để hiển thị
            seen.add(key)

# tạo 5 nhóm
group_tools = []
group_frameworks = []
group_libraries = []
group_languages = []
group_languages1 = []
group_others = []

for item in unique_items:
    key = item.lower()
    
    if key in tools:
        group_tools.append(item)
    elif key in frameworks:
        group_frameworks.append(item)
    elif key in libraries:
        group_libraries.append(item)
    elif key in code_languages:
        group_languages.append(item)
    elif key in group_languages1:
        group_languages1.append(item)
    else:
        group_others.append(item)


In [46]:
# code đọc file

# with open("data.md", "r") as f:
#     lines = f.readlines()

# print(", ".join(f"'{line.strip()}'" for line in lines))


In [47]:
# hàm tách cột thành các cột nhỏ

def split_skill_column(df, column_name, tools, frameworks, libraries, code_languages,languages):

    # ——— CHUYỂN TẤT CẢ DANH SÁCH SO SÁNH VỀ LOWER ———
    tools = {t.lower() for t in tools}
    frameworks = {f.lower() for f in frameworks}
    libraries = {l.lower() for l in libraries}
    code_languages = {c.lower() for c in code_languages}
    languages = {c.lower() for c in languages}

    # ——— HÀM XỬ LÝ CHO TỪNG DÒNG ———
    def process_row(skill_list):
        if not isinstance(skill_list, list):
            return pd.Series([np.nan]*5)

        tools_list = []
        fw_list = []
        lib_list = []
        code_lang_list = []
        lang_list = []
        
        others_list = []

        seen = set()

        for item in skill_list:
            key = item.lower()

            if key in seen:
                continue
            seen.add(key)

            if key in tools:
                tools_list.append(item)
            elif key in frameworks:
                fw_list.append(item)
            elif key in libraries:
                lib_list.append(item)
            elif key in code_languages:
                code_lang_list.append(item)
            elif key in languages:
                lang_list.append(item)
            else:
                others_list.append(item)

        # Trả về theo từng cột
        return pd.Series([
            tools_list if tools_list else np.nan,
            fw_list if fw_list else np.nan,
            lib_list if lib_list else np.nan,
            code_lang_list if code_lang_list else np.nan,
            lang_list if lang_list else np.nan,
            others_list if others_list else np.nan
        ])

    # ——— TẠO 5 CỘT MỚI ———
    df[['tools', 'frameworks', 'libraries', 'code_languages','languages', 'others']] = \
        df[column_name].apply(process_row)

    return df


In [48]:
# thực thi hàm

df = split_skill_column(
    df,
    'skills_list',
    tools,
    frameworks,
    libraries,
    code_languages,
    languages
)

In [49]:
# hàm sắp xếp thứ tự cột

def reorder_columns(df, column_order):
    other_cols = [col for col in df.columns if col not in column_order]
    new_order = column_order + other_cols
    return df[new_order]

desired_order = ['position_names', 'job_group','level','salary', 'kind_jobs', 'domain_list', 'domain_group', 'skills_list','languages','tools','frameworks','libraries','code_languages','others','company_names','city_clean','post_date']
df = reorder_columns(df, desired_order)

In [50]:
# seen = set()

# for arr in df.others:
#     if isinstance(arr, float) and math.isnan(arr):
#         continue  # bỏ qua giá trị nan
    
#     for value in arr:
#         if value not in seen:
#             print(value)
#             seen.add(value)


In [51]:
# đổi tên

df = df.rename(columns={'position_names': 'job_names'})
df = df.rename(columns={'kind_jobs': 'work_mode'})
df = df.rename(columns={'code_languages':'programming_languages' })
df = df.rename(columns={'city_clean': 'city'})

In [52]:
# đưa ra file dữ liệu sạch

df.to_csv("D:\I. KhongPhanLoai\IT-job-analysis-VN\data_clean\clean_data.csv", index=False, encoding="utf-8-sig")

<>:3: SyntaxWarning: invalid escape sequence '\I'
<>:3: SyntaxWarning: invalid escape sequence '\I'
C:\Users\My Computer\AppData\Local\Temp\ipykernel_10032\212038559.py:3: SyntaxWarning: invalid escape sequence '\I'
  df.to_csv("D:\I. KhongPhanLoai\IT-job-analysis-VN\data_clean\clean_data.csv", index=False, encoding="utf-8-sig")
